# Práctica Guiada: Datasets de tablas múltiples- Archivo ENRON

* En esta actividad se explorará un subconjunto de datos de la Base de Datos de E-mails de Enron.

* Este dataset fue preparado y recopilado por el Proyecto CALO (A Cognitive Assistant that Learns and Organizes).

* El dataset original contiene datos de unos 150 usuarios, la mayoría gerentes de Enron y aproximadamente 0,5 millones de mensajes.

* Estos datos fueron publicados en la web por la Comisión Regulatoria de Energía Federal (EEUU) durante su investigación.

## 1. Importar datos

Conectarse al archivo 'enron.db' utilizando alguno de los siguientes métidos:

- Paquete sqlite3 de Python
- pandas.read_sql
- SQLite Manager Firefox extension, DB Browser for SQLite, etc.

Observar la base de datos y consultar la tabla maestra. ¿Cuántas tablas hay en la Base de Datos?

> Respuesta:
Hay 3 tablas:
- MessageBase
- RecipientBase
- EmployeeBase

In [ ]:
import sqlite3
import pandas as pd

con = sqlite3.connect('enron.db')

data = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", con)
data

Consultar la tabla `sqlite_master` para obtener el schema de la tabla `EmployeeBase`.

1. ¿Cuántos campos tiene?
2. ¿Qué tipo de datos tiene cada uno?

In [ ]:
import sqlite3
import pandas as pd

con = sqlite3.connect('enron.db')
for row in con.execute("pragma table_info('EmployeeBase')").fetchall():
    print(row)

1. Imprimir las primeras 5 filas de la tabla EmployeeBase
2. Imprimir las primeras 5 filas de la tabla MessageBase
3. Imprimir las primeras 5 filas de la tabla RecipientBase

**Ayuda**  combinar `SELECT` con `LIMIT`.



Importar cada una de las 3 tablas a una estructura Pandas Dataframes

In [ ]:
import sqlite3
import pandas as pd

con = sqlite3.connect('enron.db')

In [ ]:
data1 = pd.read_sql_query("SELECT * FROM EmployeeBase LIMIT 5", con)
data1

In [ ]:
data2 = pd.read_sql_query("SELECT * FROM MessageBase LIMIT 5", con)
data2

In [ ]:
data3 = pd.read_sql_query("SELECT * FROM RecipientBase LIMIT 5", con)
data3

## 2. Exploración de Datos

Utilizar los 3 dataframes para responder las siguientes preguntas:

1. ¿Cuántos empleados hay en la compañía?
2. ¿Cuántos mensajes hay en la Base de Datos?
3. Algunos mensajes se enviaron a más de un destinatario. Agrupar los mensajes por message_id y contar la cantidad de destinatarios. Luego, observar la distribución de la cantidad de destinatarios.
    - ¿Cuántos mensajes tienen sólo 1 destinatario?
    - ¿Cuántos mensajes tienen al menos 5 destinatarios?
    - ¿Cuál es la máxima cantidad de destinatarios de un mensaje?
    - ¿Quién envió el mensaje con la máxima cantidad de destinatarios?

In [ ]:
import sqlite3
import pandas as pd
import datetime
import numpy as np

con = sqlite3.connect('enron.db')

data1 = pd.read_sql_query("SELECT * FROM EmployeeBase", con)
data2 = pd.read_sql_query("SELECT * FROM MessageBase", con)
data3 = pd.read_sql_query("SELECT * FROM RecipientBase", con)

In [ ]:
# 1. Versión SQL
pd.read_sql_query("SELECT count(1) FROM EmployeeBase", con)

In [ ]:
# 1. Versión Pandas
data1['eid'].count()

In [ ]:
# 2. Versión Pandas
data2['mid'].count()

In [ ]:
# 3. Cantidad de destinatarios por mensaje. Versión SQL

pd.read_sql_query("SELECT cantidad, count(1) as ocurrencias FROM \
                  (SELECT mid, count(1) as cantidad FROM RecipientBase GROUP BY mid) as mess_cant \
                  GROUP BY cantidad", con)

In [ ]:
# 3 Versión Pandas
cantidades = data3.groupby('mid').size()
cantidades.value_counts()
# Alternativa usando count() y especificando la columna a contar:
# data3.groupby('mid')['rno'].count()

In [ ]:
# 3.a Mensajes con un sólo destinatario SQL

pd.read_sql_query("SELECT cantidad, count(1) as ocurrencias FROM \
                  (SELECT mid, count(1) as cantidad FROM RecipientBase GROUP BY mid) as mess_cant \
                  WHERE cantidad = 1 \
                  GROUP BY cantidad", con)

In [ ]:
# 3.a Versión Pandas
destinatarios = data3.groupby('mid').size()
destinatarios[destinatarios == 1].count()

In [ ]:
# 3.b Mensajes con más de 5 destinatario SQL

pd.read_sql_query("SELECT count(1) as mensajes FROM \
                  (SELECT mid, count(1) as cantidad FROM RecipientBase GROUP BY mid) as mess_cant \
                  WHERE cantidad >= 5 \
                  ", con)

In [ ]:
# 3.b Versión Pandas
destinatarios[destinatarios >= 5].count()

In [ ]:
# 3.c Máxima cantidad de destinatarios SQL

pd.read_sql_query("SELECT MAX(cantidad) FROM \
                  (SELECT mid, count(1) as cantidad FROM RecipientBase GROUP BY mid) as mess_cant \
                  ", con)

In [ ]:
# 3.c
destinatarios[destinatarios == destinatarios.max()]

In [ ]:
# 3.d ¿Quién envió el mensaje más frecuente? (SQL)
pd.read_sql_query("SELECT m.mid, e.name FROM \
                  (SELECT c.mid, count(1) as cantidad FROM RecipientBase  c GROUP BY c.mid) as mess_cant \
                  INNER JOIN MessageBase as m on mess_cant.mid = m.mid \
                  INNER JOIN EmployeeBase as e on e.eid = m.from_eid \
                  WHERE cantidad = 57 \
                  ", con)

In [ ]:
# 3.d Pandas
data2[(data2['mid'] == 12116) | (data2['mid'] == 12151)]

# o bien:
data2[(data2['mid'] == 12116) | (data2['mid'] == 12151)]['from_eid']

# luego buscar el Employee = 67
data1[data1['eid'] == 67] 

# Alternativa con MERGE (simil Join) y mostrando los campos mid y name
pd.merge(data1, data2[(data2['mid'] == 12116) | (data2['mid'] == 12151)], left_on='eid', right_on='from_eid')[['mid','name']]


## 3. Combinación de datos

Utilizar la función `merge` de Pandas para combinar la información de los 3 dataframes y responder a las siguientes preguntas:

1. ¿Hay más empleados Hombres o Mujeres?
2. ¿Cómo se distribuye el género a través de los departamentos?
3. ¿Quiénes envían más e-mails, los hombres o las mujeres?
4. ¿Cuál es el promedio de e-mails enviados por cada género?
5. ¿Hay más Juniors o Seniors?
6. ¿Quiénes envían más e-mails, los Juniors o los Seniors?
7. ¿Qué departamento envía más e-mails? ¿Cómo se relaciona con la cantidad de empleados del departamento?
8. ¿Cuál es el Top 3 de enviadores de e-mails? (los 3 empleados que más e-mails envían)

In [ ]:
import sqlite3
import pandas as pd
import datetime
import numpy as np

con = sqlite3.connect('enron.db')

data1 = pd.read_sql_query("SELECT * FROM EmployeeBase", con)
data2 = pd.read_sql_query("SELECT * FROM MessageBase", con)
data3 = pd.read_sql_query("SELECT * FROM RecipientBase", con)

In [ ]:
print("1.")
if data1.groupby('gender').size()['Male']>data1.groupby('gender').size()['Female']:
    print("Hay más empleados Hombres:", data1.groupby('gender').size()['Male'])
else:
    print("Hay más empleados Mujeres (o igual que Hombres):", data1.groupby('gender').size()['Female'])

In [ ]:
print("2.")
print(data1.groupby(['gender','department']).size())


In [ ]:
print("3.")
if pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Male']>pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Female']:
    print("Los Hombres envían más e-mails:",pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Male'])
else:
    print("Las Mujeres envían más e-mails:",pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Female'])


In [ ]:
print("4.")
print("En promedio las Mujeres envían",(pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Female'] / data1.groupby('gender').size()['Female']),"e-mails")
print("En promedio los Hombres envían",(pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Male'] / data1.groupby('gender').size()['Male']),"e-mails")


In [ ]:
print("5.")
if data1.groupby('seniority').size()['Senior']>data1.groupby('seniority').size()['Junior']:
    print("Hay más empleados Senior:", data1.groupby('seniority').size()['Senior'])
else:
    print("Hay más empleados Junior :", data1.groupby('seniority').size()['Junior'])


In [ ]:
print("6.")
if pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('seniority').size()['Senior']>pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('seniority').size()['Junior']:
    print("Los Senior envían más e-mails:",pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('seniority').size()['Senior'])
else:
    print("Los Junior envían más e-mails:",pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('seniority').size()['Junior'])


In [ ]:
print("7.")
print(pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('department').size())
print(data1.groupby('department').size())


In [ ]:
print("8.")
print(data2.groupby('from_eid').size().nlargest(3, "first"))

Responder a las siguientes preguntas de aceurdo a los mensajes recibidos:

1. ¿Quiénes reciben más e-mails, los Hombres o las Mujeres?
2. ¿Quiénes reciben más e-mails, los Juniors o los Seniors?
3. ¿Qué departamento recibe más e-mails? ¿Cómo se relaciona con la cantidad de empleados del departamento?
4. ¿Cuál es el Top 5 de receptores de e-mails? (los 5 empleados que más e-mails recibieron)

In [ ]:
import sqlite3
import pandas as pd
import datetime
import numpy as np

con = sqlite3.connect('enron.db')

data1 = pd.read_sql_query("SELECT * FROM EmployeeBase", con)
data2 = pd.read_sql_query("SELECT * FROM MessageBase", con)
data3 = pd.read_sql_query("SELECT * FROM RecipientBase", con)

Hombres_reciben = pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('gender').size()['Male']
Mujeres_reciben = pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('gender').size()['Female']

Junior_reciben = pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('seniority').size()['Junior']
Senior_reciben = pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('seniority').size()['Senior']

In [ ]:
print("1.")
if Hombres_reciben>Mujeres_reciben:
    print("Los Hombres reciben más e-mails")
else:
    print("Las Mujeres reciben más e-mails (o igual)")

In [ ]:
print ("2.")
if Junior_reciben>Senior_reciben:
    print("Los Junior reciben más e-mails")
else:
    print("Las Senior reciben más e-mails (o igual)")


In [ ]:
print("3.")
print(pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('department').size())
print(data1.groupby('department').size())


In [ ]:
print("4.")
print(data3.groupby('to_eid').size().nlargest(5, "first"))


¿Cuáles empleados enviaron más e-mails de forma "masiva"?

In [ ]:
import sqlite3
import pandas as pd
import datetime
import numpy as np

con = sqlite3.connect('enron.db')

data1 = pd.read_sql_query("SELECT * FROM EmployeeBase", con)
data2 = pd.read_sql_query("SELECT * FROM MessageBase", con)
data3 = pd.read_sql_query("SELECT * FROM RecipientBase", con)

enviadores = pd.merge(data2, data3, on='mid').groupby(['from_eid','mid']).size()
print(enviadores[enviadores > 50]) ## Los eid 67 y 68 son los que más "spam" hicieron




Continuar explorando el dataset. ¿Qué otras preguntas se podrían hacer?

Trabajar en grupos de a 2. Intercambiando preguntas desafiantes para que el otro las responda.